In [1]:
import pandas as pd
from src.dimension_modeler import Dimension_Modeler

### DIM PRODUCTS
- product_id (int) (Natural Key)
- product_key (int) (Surrogate Key)
- product_name (str)
- product_category (str)
- product_price (decimal 10,2)
- product_cost (decimal 10,2)
- is_current(boolean)
- start_date (date)
- end_date (date)

In [9]:
products_df = pd.read_csv('datasets/donut_shop_product_info.csv')

In [10]:
prod_dim_model = Dimension_Modeler(products_df)

In [12]:
prod_dim_model.df.head()

,Product Category,Product Price,Product Cost,Historical Cost
0,Donut,1,0.10,0.1
1,Hot Foods,3,0.80,0.5
2,Drinks,2,1.15,1.0


In [13]:
transactions_df = pd.read_csv('datasets/donut_shop_transactions.csv')

In [77]:
price_history = []


In [ ]:
for x in transactions_df.groupby('Category')['Price per Unit'].value_counts().index:
    price_history.append({
            'category': x[0],
            'price': x[1],
            'start_date': transactions_df[transactions_df['Price per Unit'] == x[1]]['Date'].min(),
            'end_date': transactions_df[transactions_df['Price per Unit'] == x[1]]['Date'].max(),
            }
                         )

In [80]:
price_history

[{'category': 'Donut',
  'price': 1.0,
  'start_date': '2020-01-01 00:27:34',
  'end_date': '2024-12-31 23:53:27'},
 {'category': 'Donut',
  'price': 0.5,
  'start_date': '2015-01-01 01:35:44',
  'end_date': '2019-12-31 23:49:08'},
 {'category': 'Drinks',
  'price': 2.0,
  'start_date': '2020-01-01 03:05:09',
  'end_date': '2024-12-31 23:53:27'},
 {'category': 'Drinks',
  'price': 1.25,
  'start_date': '2015-01-01 00:56:32',
  'end_date': '2019-12-31 23:49:08'},
 {'category': 'Hot Foods',
  'price': 3.0,
  'start_date': '2020-01-01 00:27:34',
  'end_date': '2024-12-31 23:53:27'},
 {'category': 'Hot Foods',
  'price': 1.5,
  'start_date': '2015-01-01 01:41:45',
  'end_date': '2019-12-31 23:32:46'}]

In [19]:
transactions_df[transactions_df['Price per Unit'] == 1.00]['Date'].max()

'2024-12-31 23:53:27'

In [20]:
transactions_df[transactions_df['Price per Unit'] == 1.00]['Date'].min()

'2020-01-01 00:27:34'

In [ ]:
products_df[]

### DIM EMPLOYEES
- employee_id (int) DONE 
- employee_name (str) DONE 
- job_title (str) DONE 

In [2]:
employee_df = pd.read_csv('datasets/donut_shop_employees.csv')


In [3]:
dim_model = Dimension_Modeler(employee_df)

In [4]:
employee_df.columns = ['employee_name', 'job_title']

In [5]:
employee_df.dtypes

employee_name    object
job_title        object
dtype: object

In [6]:
employee_df = dim_model.make_id_col()

In [7]:
employee_df.dtypes

id                int64
employee_name    object
job_title        object
dtype: object

In [8]:
dim_model.make_csv('dim_employees')

dim_employees.csv created in datasets/dimensions/
